In [ ]:
import os
import re
import copy
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.modeling import models, fitting
from astropy.wcs import WCS
import matplotlib.animation as animation
from astropy.stats import sigma_clipped_stats
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry
from photutils.background import Background2D, MedianBackground
from photutils import DAOStarFinder
from astropy.stats import mad_std
from cosmos_helper_funcs import get_stacks, label_plot, cosmos_gain_dict, get_mean_images
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.coordinates import match_coordinates_sky
from astropy.table import Table
from scipy.optimize import curve_fit
import scipy.io

data_folder = '/Volumes/Extreme SSD/COSMOS data'

In [ ]:
data_file = '/Users/layden/Library/CloudStorage/Box-Box/Scientific CMOS - MKI ONLY (might contain EAR; ITAR)/Teledyne_COSMOS/Analysis Images/On Sky Data/calibrated_with_WCS.fits'
data_file_2 = '/Users/layden/Library/CloudStorage/Box-Box/Scientific CMOS - MKI ONLY (might contain EAR; ITAR)/Teledyne_COSMOS/Analysis Images/On Sky Data/calibrated_CMS_RS_M38_r_filter1000ms_-25C 2024 November 01 08_20_10.fits'
# data_file_2 = '/Users/layden/Library/CloudStorage/Box-Box/Scientific CMOS - MKI ONLY (might contain EAR; ITAR)/Teledyne_COSMOS/Analysis Images/On Sky Data/Uncalibrated/CMS_RS_M38_r_filter1000ms_-25C 2024 November 01 08_20_10.fits'
dark_file = '/Users/layden/Library/CloudStorage/Box-Box/Scientific CMOS - MKI ONLY (might contain EAR; ITAR)/Teledyne_COSMOS/Analysis Images/On Sky Data/calibrated_CMS_RS_dark_1000ms_-25C 2024 October 31 21_36_05.fits'
defect_pix_file_1 = '/Users/layden/Library/CloudStorage/Box-Box/Scientific CMOS - MKI ONLY (might contain EAR; ITAR)/Teledyne_COSMOS/Final Data Products/Calibration/Defect Maps/defect_map_calibration_CMS.fits'
defect_pix_file_2 = '/Users/layden/Library/CloudStorage/Box-Box/Scientific CMOS - MKI ONLY (might contain EAR; ITAR)/Teledyne_COSMOS/Final Data Products/Calibration/Defect Maps/defect_map_dark_CMS.fits'
hdu = fits.open(data_file)[0]
wcs = WCS(hdu.header)
data = hdu.data
hdu2 = fits.open(data_file_2)[0]
data2 = hdu2.data.astype(int)
data = data2[1]
# data = np.mean(data2, axis=0)
dark_data = fits.getdata(dark_file).astype(int)
dark_data_avg = np.mean(dark_data, axis=0)
data = data - dark_data_avg
bkg_sigma = mad_std(data)  # Background standard deviation
defect_data_1 = fits.getdata(defect_pix_file_1)
defect_data_2 = fits.getdata(defect_pix_file_2)
defect_data_1[1257] = np.NaN
defect_data_1[6451] = np.NaN
defect_data_1[:,994] = np.NaN
data = data * defect_data_1 * defect_data_2
# Print the pixel scale
print(wcs.pixel_scale_matrix * 3600)
print(np.nanmedian(data))

In [ ]:
# Make the flat field correction
flat_field_file = '/Users/layden/Library/CloudStorage/Box-Box/Scientific CMOS - MKI ONLY (might contain EAR; ITAR)/Teledyne_COSMOS/Analysis Images/On Sky Data/calibrated_CMS_RS_SkyFlats_1000ms_-25_r_filter 2024 October 31 21_19_20.fits'
flat_field_data = fits.getdata(flat_field_file)
flat_field_mean_img = np.mean(flat_field_data, axis=0)
flat_field = (flat_field_mean_img - dark_data_avg) * defect_data_1 * defect_data_2
# Divide by the upper 0.5 percentile to normalize the flat field without outliers skewing the result
flat_field = flat_field / np.nanpercentile(flat_field, 99.5)
data = data / flat_field
# Turn NaNs into background value
print(np.nanmedian(data))
data = np.nan_to_num(data, nan=np.nanmedian(data))
data = data + 100

In [ ]:
daofind = DAOStarFinder(fwhm=3.0, threshold=100)
sources = daofind(data)
positions = [(x, y) for x, y in zip(sources['xcentroid'], sources['ycentroid'])]
# Define aperture and annulus radii
aperture_radius = 6  # pixels
annulus_inner_radius = 18  # pixels
annulus_outer_radius = 21  # pixels

# Create apertures and annuli
apertures = CircularAperture(positions, r=aperture_radius)
annuli = CircularAnnulus(positions, r_in=annulus_inner_radius, r_out=annulus_outer_radius)

# Perform aperture photometry on the image
aperture_photometry_result = aperture_photometry(data, apertures)

# Perform background photometry using annuli
annulus_masks = annuli.to_mask(method='center')
background_medians = []

for mask in annulus_masks:
    annulus_data = mask.multiply(data)
    annulus_data_1d = annulus_data[mask.data > 0]  # Exclude zero-padding
    # Ignore warning for clipping NaNs
    _, median_background, _ = sigma_clipped_stats(annulus_data_1d, sigma=3.0)
    background_medians.append(median_background)

# Subtract background and calculate final flux
aperture_sums = aperture_photometry_result['aperture_sum']
background_sums = np.array(background_medians) * apertures.area
final_fluxes = aperture_sums - background_sums

# Add results to the photometry table
aperture_photometry_result['RA'], aperture_photometry_result['DEC'] = wcs.all_pix2world(aperture_photometry_result['xcenter'], aperture_photometry_result['ycenter'], 0)
aperture_photometry_result['background'] = background_medians
aperture_photometry_result['final_flux'] = final_fluxes
# aperture_photometry_result = aperture_photometry_result[~np.isnan(aperture_photometry_result['final_flux'])]
aperture_photometry_result.sort('final_flux')
# Only keep those with positive final flux
aperture_photometry_result = aperture_photometry_result[aperture_photometry_result['final_flux'] > 0]
# Exclude stars within 50 pixels of the edge
aperture_photometry_result = aperture_photometry_result[(aperture_photometry_result['xcenter'] / u.pix > 50)]
aperture_photometry_result = aperture_photometry_result[(aperture_photometry_result['ycenter'] / u.pix > 50)]
aperture_photometry_result = aperture_photometry_result[(aperture_photometry_result['xcenter'] / u.pix < 8070)]
aperture_photometry_result = aperture_photometry_result[(aperture_photometry_result['ycenter'] / u.pix < 8070)]

# Print the results
print(aperture_photometry_result)

# # Exclude stars that are within 30 pixels of each other
# # Find distance to nearest star for each star
# distances = np.zeros(len(aperture_photometry_result))
# for i, entry in enumerate(aperture_photometry_result):
#     xpos = entry['xcenter']
#     ypos = entry['ycenter']
#     min_dist = 100
#     for j, entry2 in enumerate(aperture_photometry_result):
#         if i == j:
#             continue
#         xpos2 = entry2['xcenter']
#         ypos2 = entry2['ycenter']
#         distance = np.sqrt((xpos - xpos2) ** 2 + (ypos - ypos2) ** 2) / u.pix
#         if distance < min_dist:
#             min_dist = distance
#     distances[i] = min_dist
#     print(distances[i])

# apertures = CircularAperture(positions, r=5)  # 5-pixel radius aperture
# phot_table = aperture_photometry(data, apertures)
# phot_table['RA'], phot_table['DEC'] = wcs.all_pix2world(phot_table['xcenter'], phot_table['ycenter'], 0)
# # Drop any stars with NaNs
# phot_table = phot_table[~np.isnan(phot_table['aperture_sum'])]
# print(phot_table)

In [ ]:
image_center = SkyCoord.from_pixel(hdu.data.shape[1] / 2, hdu.data.shape[0] / 2, wcs)
print(f"Image center (RA, Dec): {image_center.ra.deg}, {image_center.dec.deg}")
# Use the center position for querying the catalog
radius = 60 * u.arcmin  # Set your desired search radius

# Query the star catalog
Vizier.ROW_LIMIT = -1  # Remove row limit for large queries
result = Vizier.query_region(
    image_center, radius=radius, catalog='I/345/gaia2')[0]

# Only keep objects with Gmag below 17.5
result = result[result['Gmag'] < 17.5]
# Convert every star in result to an x and y position on COSMOS
coords = SkyCoord(result['RA_ICRS'], result['DE_ICRS'], unit='deg')
x, y = wcs.all_world2pix(coords.ra, coords.dec, 0)
result['x'] = x
result['y'] = y
# Only keep stars that are within the image
result = result[(result['x'] >= 0) & (result['x'] < hdu.data.shape[1]) &
                (result['y'] >= 0) & (result['y'] < hdu.data.shape[0])]
# print(result['RA_ICRS', 'DE_ICRS', 'Gmag', 'x', 'y'])
print(result)

In [ ]:
# Find the proximity to the nearest star for each star in the catalog
distances = np.zeros(len(result))
distance_cutoff = 35
for i, entry in enumerate(result):
    xpos = entry['x']
    ypos = entry['y']
    min_dist = 100
    for j, entry2 in enumerate(result):
        if i == j:
            continue
        xpos2 = entry2['x']
        ypos2 = entry2['y']
        distance = np.sqrt((xpos - xpos2) ** 2 + (ypos - ypos2) ** 2)
        if distance < min_dist:
            min_dist = distance
        if min_dist < distance_cutoff:
            break
    distances[i] = min_dist
    print(i, distances[i])
# Make distances a row in the table
result['distance'] = distances
# Remove stars that are too close to each other
result = result[result['distance'] > distance_cutoff]

In [ ]:
# If using Gaia
# Match the photometry table with the star catalog
coords = SkyCoord(ra=aperture_photometry_result['RA'] * u.deg, dec=aperture_photometry_result['DEC'] * u.deg)
coords_catalog = SkyCoord(ra=result['RA_ICRS'], dec=result['DE_ICRS'])
idx, d2d, d3d = match_coordinates_sky(coords, coords_catalog)
# Filter matches with a threshold (e.g., 1 arcsecond)
threshold = 2 * u.arcsec
matched = d2d < threshold
matched_catalog = Table({
                    'RA': result['RA_ICRS'][idx[matched]],
                    'Dec': result['DE_ICRS'][idx[matched]],
                    'Gmag': result['Gmag'][idx[matched]],
                    'e_Gmag': result['e_Gmag'][idx[matched]],
                    'xcentroid': aperture_photometry_result['xcenter'][matched],
                    'ycentroid': aperture_photometry_result['ycenter'][matched],
                    'ra_image': aperture_photometry_result['RA'][matched],
                    'dec_image': aperture_photometry_result['DEC'][matched],
                    'aperture_sum': aperture_photometry_result['aperture_sum'][matched],
                    'meas_flux': aperture_photometry_result['final_flux'][matched],
                    'separation': d2d[matched].arcsec,
                    'gaia_id': result['Source'][idx[matched]],
                    'BP-RP': result['BP-RP'][idx[matched]]
                    })
matched_catalog.sort('meas_flux')

print(matched_catalog)


In [ ]:
wavelengths = np.arange(336, 1022, 2)
sloan_r_file = '/Users/layden/Downloads/SLOAN.SDSS.rprime_filter.xml'
sloan_r = Table.read(sloan_r_file)
# Convert to numpy array
sloan_r = sloan_r.to_pandas()
sloan_r['Wavelength'] /= 10
# Interpolate the filter to find values at elements of wavelengths
sloan_r_interp = np.interp(wavelengths, sloan_r['Wavelength'], sloan_r['Transmission'])
cosmos_arr = np.genfromtxt('/Users/layden/Documents/Image-Sensor-Testing/data/cosmos_qe_datasheet.csv', delimiter=',')
cosmos_arr_interp = np.interp(wavelengths, cosmos_arr[:, 0], cosmos_arr[:, 1])
plt.plot(wavelengths, sloan_r_interp)
plt.plot(wavelengths, cosmos_arr_interp)
plt.show()

In [ ]:
# Use Gaia spectra to calculate expected flux with our filter and QE
filename = '/Users/layden/Downloads/XpSampledMeanSpectrum_020748-020984.csv'
import pandas as pd
# Load csv into pandas dataframe
col_names = ['source_id', 'solution_id', 'ra', 'dec', 'flux', 'flux_err']
# Use 100 rows
df = pd.read_csv(filename, skiprows=64, names=col_names)
source_ids = df['source_id'].to_numpy()
# Want to convert Gaia fluxes to phot/m^2/nm/s. They're currently in W/m^2/nm
h = 6.62607015e-34  # Planck's constant in J s
c = 299792458  # Speed of light in m/s
# Create column in matched_catalog for fluxes
matched_catalog['gaia_phot_flux'] = np.zeros(len(matched_catalog), dtype=object)
matched_catalog['gaia_phot_flux_err'] = np.zeros(len(matched_catalog), dtype=object)
for i, star in enumerate(matched_catalog):
    idx = np.where(source_ids == star['gaia_id'])[0]
    if len(idx) > 0:
        fluxes = df['flux'][idx[0]]
        flux_errs = df['flux_err'][idx[0]]
        # Convert fluxes string into numpy array. First remove brackets
        fluxes = fluxes[1:-1].split(',')
        flux_errs = flux_errs[1:-1].split(',')
        fluxes = np.array(fluxes, dtype=float)
        flux_errs = np.array(flux_errs, dtype=float)
        avg_flux_err = np.mean(flux_errs / fluxes)
        phot_fluxes = fluxes / (h * c / (wavelengths * 1e-9))
        phot_fluxes = phot_fluxes * sloan_r_interp * cosmos_arr_interp
        tot_phot_flux = np.sum(phot_fluxes)
        matched_catalog['gaia_phot_flux'][i] = tot_phot_flux
        matched_catalog['gaia_phot_flux_err'][i] = abs(avg_flux_err)
# Exclude stars with no Gaia spectra
matched_catalog = matched_catalog[matched_catalog['gaia_phot_flux'] > 0]
matched_catalog = matched_catalog[matched_catalog['gaia_phot_flux_err'] < 0.1]
print(matched_catalog)

In [ ]:
mean_vs_multiplier = np.genfromtxt('mean_vs_multiplier.csv', delimiter=',', skip_header=1)
mean_vals = mean_vs_multiplier[:, 0]
multiplier_vals = mean_vs_multiplier[:, 1]
gain = 4.3
raw_read_noise = 0.8
raw_bkg_level = 1.0 # e-
# Use mean image to figure out how much shot noise should be multiplied to account
# for effective noise due to nonlinearity
matched_catalog_positions = [(x, y) for x, y in zip(matched_catalog['xcentroid'], matched_catalog['ycentroid'])]
shot_noise_vals = np.zeros(len(matched_catalog_positions))
read_noise_vals = np.zeros(len(matched_catalog_positions))
bkg_noise_vals = np.zeros(len(matched_catalog_positions))
for i, position in enumerate(matched_catalog_positions):
    star_subarray = data[int(position[1]) - 10:int(position[1]) + 10,
                         int(position[0]) - 10:int(position[0]) + 10] - 100
    multiplier_subarray = np.interp(star_subarray, mean_vals, multiplier_vals)
    var_shotnoise_subarray = gain * star_subarray * multiplier_subarray
    var_readnoise_subarray = (raw_read_noise * gain * multiplier_subarray) ** 2
    var_bkgnoise_subarray = (raw_bkg_level * gain * multiplier_subarray) ** 2
    aperture = CircularAperture([10,10], r=aperture_radius)
    shot_var = aperture_photometry(var_shotnoise_subarray, aperture)['aperture_sum'][0]
    read_var = aperture_photometry(var_readnoise_subarray, aperture)['aperture_sum'][0]
    bkg_var = aperture_photometry(var_bkgnoise_subarray, aperture)['aperture_sum'][0]
    shot_noise_vals[i] = np.sqrt(shot_var)
    read_noise_vals[i] = np.sqrt(read_var)
    bkg_noise_vals[i] = np.sqrt(bkg_var)
# PTC noise vals are the standard deviation in e- expected corresponding to stars in the matched catalog
# Get rid of outliers in the PTC noise values and gaia flux values
start_val = 0
# plt.plot(matched_catalog['gaia_phot_flux'][start_val:], ptc_noise_vals[start_val:], 'o')
# plt.xscale('log')
# plt.yscale('log')
# plt.plot(plot_points, ptc_noise_vals_interp, '-')
indices_to_exclude = [694]
# indices_to_exclude = [694, 730, 731, 735, 736, 737, 738, 739, 742, 743, 744, 745]
clean_shot_noise_vals = np.delete(shot_noise_vals, indices_to_exclude)
clean_read_noise_vals = np.delete(read_noise_vals, indices_to_exclude)
clean_bkg_noise_vals = np.delete(bkg_noise_vals, indices_to_exclude)
clean_gaia_phot_flux = np.delete(matched_catalog['gaia_phot_flux'].data.astype(np.float64), indices_to_exclude)
clean_e_flux_vals = np.delete(matched_catalog['meas_flux'].data.astype(np.float64) / gain, indices_to_exclude)
plt.plot(clean_e_flux_vals, clean_shot_noise_vals, '-')
plt.plot(clean_e_flux_vals, clean_read_noise_vals, '-')
plt.plot(clean_e_flux_vals, clean_bkg_noise_vals, '-')
plt.show()

In [ ]:
%matplotlib inline
# Increase resolution for plots
plt.rcParams['figure.dpi'] = 200
gain = 4.3
matched_catalog['meas_e_flux'] = matched_catalog['meas_flux'] / gain
# Convert Gmag to flux
matched_catalog['flux'] = 10 ** (-0.4 * (matched_catalog['gaia_phot_flux'] - 25.688365))
# plt.plot(matched_catalog['Gmag'], line(matched_catalog['Gmag'], *popt2), color='red')
# plt.ylim(0, 0.002)
# plt.show()

fig, ax = plt.subplots(1, 1, figsize=(6, 4.5))
plot_points = np.linspace(np.min(matched_catalog['gaia_phot_flux']), np.max(matched_catalog['gaia_phot_flux']), 1000)
# Increase font size
plt.rcParams.update({'font.size': 13})
popt, _ = curve_fit(lambda x, a, b: a * x + b, matched_catalog['gaia_phot_flux'], matched_catalog['meas_e_flux'])
# ax.plot(matched_catalog['gaia_phot_flux'], matched_catalog['meas_e_flux'], 'o', markersize=2, label='Measured Data')
# Make label for best fit line have slope
ax.plot(plot_points, np.polyval(popt, plot_points), 'r--', label=f'Best fit line, slope = {popt[0]:.2f} $m^2$')
ax.set_xlabel(r'Expected signal flux ($e^-/m^2/s$)', fontsize=14)
ax.set_ylabel(r'Measured signal ($e^-/s$)', fontsize=14)
ax.set_xscale('log')
ax.set_yscale('log')
read_noise_vals_interp = np.interp(matched_catalog['gaia_phot_flux'].data.astype(np.float64), clean_gaia_phot_flux, clean_read_noise_vals / gain)
shot_noise_vals_interp = np.interp(matched_catalog['gaia_phot_flux'].data.astype(np.float64), clean_gaia_phot_flux, clean_shot_noise_vals / gain)
bkg_noise_vals_interp = np.interp(matched_catalog['gaia_phot_flux'].data.astype(np.float64), clean_gaia_phot_flux, clean_bkg_noise_vals / gain)
scint_noise_vals = np.polyval(popt, matched_catalog['gaia_phot_flux'].data.astype(np.float64)) * 0.005
tot_noise_vals = np.sqrt(read_noise_vals_interp ** 2 + shot_noise_vals_interp ** 2 + bkg_noise_vals_interp ** 2 + scint_noise_vals ** 2)
# tot_noise_vals_percent = tot_noise_vals / func(x, *popt) * 100
# tot_noise_vals_percent_2 = tot_noise_vals_2 / func(x, *popt) * 100
# ax.plot(plot_points, np.polyval(popt, plot_points) + tot_noise_vals, 'k:', label='Expected Noise')
# ax.plot(plot_points, np.polyval(popt, plot_points) - tot_noise_vals, 'k:')
ax.errorbar(matched_catalog['gaia_phot_flux'], matched_catalog['meas_e_flux'],
            xerr=matched_catalog['gaia_phot_flux_err'] * matched_catalog['gaia_phot_flux'],
            yerr=tot_noise_vals, fmt='o', markersize=0.5, linewidth=0.2, label='Measured data', zorder=0)
plt.legend(fontsize=11, loc='lower right')
plt.xlim(3e3, 1e6)
plt.ylim(4e2, 1e5)
# Increase padding between subplots
plt.tight_layout(pad=2.0)
plt.show()

In [ ]:
# Take the 10 brightest stars. Make a 50x50 cutout around those stars and fit to a 2D Gaussian. Report the FWHM
brightest_stars = matched_catalog[-100:]
cutouts = []
for star in brightest_stars:
    xpos = star['xcentroid']
    ypos = star['ycentroid']
    cutout = data[int(ypos) - 25:int(ypos) + 25, int(xpos) - 25:int(xpos) + 25]
    cutouts.append(cutout)

def gaussian_2d(xy, amplitude, xo, yo, sigma_x, sigma_y, theta, offset):
    """
    2D Gaussian function.
    """
    x, y = xy
    xo = float(xo)
    yo = float(yo)
    a = (np.cos(theta)**2) / (2*sigma_x**2) + (np.sin(theta)**2) / (2*sigma_y**2)
    b = -(np.sin(2*theta)) / (4*sigma_x**2) + (np.sin(2*theta)) / (4*sigma_y**2)
    c = (np.sin(theta)**2) / (2*sigma_x**2) + (np.cos(theta)**2) / (2*sigma_y**2)
    g = offset + amplitude * np.exp(-(a * (x - xo)**2 + 2*b * (x - xo) * (y - yo) + c * (y - yo)**2))
    return g.ravel()

def fit_gaussian_and_get_fwhm(cutout):
    """
    Fit a 2D Gaussian to a cutout and calculate the FWHM.
    """
    # Create coordinate grid
    x = np.arange(cutout.shape[1])
    y = np.arange(cutout.shape[0])
    x, y = np.meshgrid(x, y)
    
    # Initial guesses for fitting
    amplitude = cutout.max() - cutout.min()
    xo, yo = np.unravel_index(np.argmax(cutout), cutout.shape)
    sigma_x = sigma_y = 3.0  # Initial guess for sigma
    theta = 0.0  # Assume no rotation
    offset = cutout.min()
    initial_guess = (amplitude, xo, yo, sigma_x, sigma_y, theta, offset)
    
    try:
        # Fit the 2D Gaussian
        popt, _ = curve_fit(gaussian_2d, (x, y), cutout.ravel(), p0=initial_guess)
    except RuntimeError:
        return None  # Fit failed

    # Extract fitted parameters
    _, _, _, sigma_x, sigma_y, _, _ = popt

    # Calculate FWHM: FWHM = 2.355 * sigma
    fwhm_x = 2.355 * sigma_x
    fwhm_y = 2.355 * sigma_y
    if abs(fwhm_x) > 10 or abs(fwhm_y) > 10:
        return None

    return fwhm_x, fwhm_y

fwhm_results = np.zeros((len(cutouts), 2))
for i, cutout in enumerate(cutouts):
    result = fit_gaussian_and_get_fwhm(cutout)
    if result:
        fwhm_results[i] = result
    else:
        fwhm_results[i] = (None, None)

plt.plot(fwhm_results[:,0], 'o')
plt.plot(fwhm_results[:,1], 'o')
plt.show()
avg_fwhm = np.nanmean(fwhm_results) * wcs.pixel_scale_matrix[0, 0] * 3600
std_fwhm = np.nanstd(fwhm_results) * wcs.pixel_scale_matrix[0, 0] * 3600
print("FWHM: ", format(avg_fwhm, '3.2f'), "plus/minus", format(std_fwhm, '3.3f'), "arcsec")

In [13]:
def shift_values(arr, del_x, del_y):
    '''Shift values in an array by a specified discrete displacement.
    
    Parameters
    ----------
    arr : array-like
        The intensity grid.
    del_x : int
        The x displacement, in subpixels.
    del_y : int
        The y displacement, in subpixels.
        
    Returns
    -------
    new_arr : array-like
        The shifted array.
    '''
    m, n = arr.shape
    new_arr = np.zeros_like(arr)
    # print(abs(del_x) > m, abs(del_y) > n)
    new_arr[max(del_y, 0):m+min(del_y, 0), max(del_x, 0):n+min(del_x, 0)] = \
        arr[-min(del_y, 0):m-max(del_y, 0), -min(del_x, 0):n-max(del_x, 0)]
    return new_arr

In [ ]:
source_folder = '/Users/layden/Library/CloudStorage/Box-Box/Scientific CMOS - MKI ONLY (might contain EAR; ITAR)/Teledyne_COSMOS/Analysis Images/On Sky Data/Light Curve'
stacks = get_stacks(source_folder)[:]

In [ ]:
# Concatenate imagestacks into a single array
imagestack = np.concatenate([stack['imagestack'] for stack in stacks], axis=0)
print(0)
imagestack = imagestack.astype(np.float32)
print(1)
# Remove defect pixels
imagestack = imagestack * defect_data_1 * defect_data_2
print(2)

In [16]:
subarray_size = 200
start_x, start_y = 1150, 3699
shift_x, shift_y = 0, 0

In [ ]:
test_cube = np.zeros((len(imagestack), 200, 200))
shifts = np.zeros((len(imagestack), 2))
for i in range(len(imagestack)):
    print('Starting Frame', i)
    frame = imagestack[i]
    start_x -= shift_x
    start_y -= shift_y
    # Clear previous stack_data from memory
    # subarray_dark = dark_data_avg[start_y - subarray_size // 2:start_y + subarray_size // 2,
    #                            start_x - subarray_size // 2:start_x + subarray_size // 2]
    # subarray_flat = flat_field[start_y - subarray_size // 2:start_y + subarray_size // 2,
    #                            start_x - subarray_size // 2:start_x + subarray_size // 2]
    subarray_data = frame[start_y - subarray_size // 2:start_y + subarray_size // 2,
                        start_x - subarray_size // 2:start_x + subarray_size // 2]
    subarray_data = np.nan_to_num(subarray_data, nan=106)
    # subarray_data = (subarray_data - subarray_dark) / subarray_flat
    max_index = np.argmax(subarray_data)
    max_coords = np.unravel_index(max_index, subarray_data.shape)
    
    y, x = np.mgrid[:subarray_data.shape[0], :subarray_data.shape[1]]
    initial_guess = models.Gaussian2D(amplitude=subarray_data.max(), x_mean=max_coords[1], y_mean=max_coords[0], x_stddev=3, y_stddev=3)
    fitter = fitting.LevMarLSQFitter()
    fitted_gaussian = fitter(initial_guess, x, y, subarray_data)
    shift_x = np.rint(subarray_size / 2 - fitted_gaussian.x_mean.value).astype(int)
    shift_y = np.rint(subarray_size / 2 - fitted_gaussian.y_mean.value).astype(int)
    print(start_x, start_y, shift_x, shift_y)
    shifted_frame = shift_values(subarray_data, shift_x, shift_y)
    test_cube[i] = shifted_frame
    shifts[i] = (start_x - shift_x, start_y - shift_y)
shifts = (shifts - shifts[0]).astype(int)

In [ ]:
for i in range (len(imagestack)):
    print('Starting Frame', i)
    frame = (imagestack[i] - dark_data_avg) / flat_field + 100
    frame = np.nan_to_num(frame, nan=np.nanmedian(frame))
    frame = shift_values(frame, -shifts[i, 0], -shifts[i, 1])
    imagestack[i] = frame
mean_shifted_img = np.mean(imagestack, axis=0)

In [19]:
# %matplotlib qt
# plt.imshow(mean_shifted_img, cmap='gray', norm='log')
# plt.show()

In [ ]:
# Some weird shifts happened, so we need to shift the positions
# positions = [(x / u.pix - 16, y / u.pix + 3) for x, y in zip(aperture_photometry_result['xcenter'], aperture_photometry_result['ycenter'])]
# positions = [(x - 16, y + 3) for x, y in zip(matched_catalog['xcentroid'], matched_catalog['ycentroid'])]
daofind = DAOStarFinder(fwhm=5.0, threshold=100)
sources_mean_img = daofind(mean_shifted_img)
positions = [(x, y) for x, y in zip(sources_mean_img['xcentroid'], sources_mean_img['ycentroid'])]
# Perform aperture photometry on each frame in the image cube
light_curves_low = np.zeros((len(positions), len(imagestack)))
gain = 4.3
apertures = CircularAperture(positions, r=aperture_radius)
annuli = CircularAnnulus(positions, r_in=annulus_inner_radius, r_out=annulus_outer_radius)
# Perform background photometry using annuli
annulus_masks = annuli.to_mask(method='center')

for i, frame in enumerate(imagestack[:]):
    print('Starting Frame', i)
    img_frame = frame / gain
    # Ignore warning for clipping NaNs
    background_medians = []
    for mask in annulus_masks:
        annulus_data = mask.multiply(img_frame)
        annulus_data_1d = annulus_data[mask.data > 0]  # Exclude zero-padding
        # Ignore warning for clipping NaNs
        _, median_background, _ = sigma_clipped_stats(annulus_data_1d, sigma=3.0)
        background_medians.append(median_background)
    aperture_phot_result = aperture_photometry(img_frame, apertures)
    aperture_sums = aperture_phot_result['aperture_sum']
    background_sums = np.array(background_medians) * apertures.area
    final_fluxes = aperture_sums - background_sums
    light_curves_low[:, i] = final_fluxes

In [21]:
# mean_vs_var = np.genfromtxt('mean_vs_var.csv', delimiter=',', skip_header=1)
# mean_vals = mean_vs_var[:, 0]
# var_vals = mean_vs_var[:, 1]
# gain = 4.3
# # Use mean image to figure out how much shot noise should be multiplied to account
# # for effective noise due to nonlinearity
# ptc_noise_vals_2 = np.zeros(len(positions))
# for i, position in enumerate(positions):
#     star_subarray = imagestack[0][int(position[1]) - 10:int(position[1]) + 10,
#                          int(position[0]) - 10:int(position[0]) + 10] - 100
#     shot_noise_subarray = np.interp(star_subarray / gain, mean_vals, var_vals)
#     aperture = CircularAperture([10,10], r=aperture_radius)
#     # phot_result = aperture_photometry(star_subarray, aperture)['aperture_sum'][0]
#     noise_result = aperture_photometry(shot_noise_subarray, aperture)['aperture_sum'][0]
#     ptc_noise_vals_2[i] = np.sqrt(noise_result)

In [ ]:
%matplotlib inline
# Concatenate the light curves
mean_bkg = 5 # ADU
aper_size = np.pi * aperture_radius ** 2
avg_brightnesses = np.mean(light_curves_low, axis=0) / np.mean(light_curves_low)
light_curves_2 = light_curves_low / avg_brightnesses
light_curve_means = np.mean(light_curves_2, axis=1)
light_curve_errs = np.std(light_curves_2, axis=1) / light_curve_means
plt.plot(light_curve_means, light_curve_errs, 'o', markersize=1, label='Measured data')
mean_points = np.logspace(1, 6.3, 100)
shot_noise_vals_interp = np.interp(mean_points, clean_e_flux_vals, clean_shot_noise_vals / gain)
read_noise_vals_interp = np.interp(mean_points, clean_e_flux_vals, clean_read_noise_vals / gain)
bkg_noise_vals_interp = np.interp(mean_points, clean_e_flux_vals, clean_bkg_noise_vals / gain)
scint_noise_points = 0.005 * mean_points
tot_noise_points = np.sqrt(shot_noise_vals_interp ** 2 + read_noise_vals_interp ** 2 + scint_noise_points ** 2 + bkg_noise_vals_interp ** 2) / mean_points
naive_shotnoise_points = np.sqrt(mean_points)
naive_readnoise_points = 0.8 * np.sqrt(aper_size)
naive_bkg_points = np.sqrt(aper_size * mean_bkg)
tot_naive_noise_points = np.sqrt(naive_shotnoise_points ** 2 + naive_readnoise_points ** 2 + naive_bkg_points ** 2 + scint_noise_points ** 2)
plt.plot(mean_points, shot_noise_vals_interp / mean_points, 'g--', label='Effective shot noise')
plt.plot(mean_points, read_noise_vals_interp / mean_points, 'r-.', label='Effective read noise')
plt.plot(mean_points, bkg_noise_vals_interp / mean_points, 'b:', label='Effective background noise')
plt.plot(mean_points, scint_noise_points / mean_points, ':', color='magenta', label='Scintillation noise')
plt.plot(mean_points, tot_noise_points, 'k-', label='Total expected noise')
plt.plot(mean_points, naive_shotnoise_points / mean_points, 'g--', label='Ideal shot noise', alpha=0.2)
plt.plot(mean_points, naive_readnoise_points / mean_points, 'r-.', label='Ideal read noise', alpha=0.2)
plt.plot(mean_points, naive_bkg_points / mean_points, 'b:', label='Ideal background noise', alpha=0.2)
plt.plot(mean_points, tot_naive_noise_points / mean_points, 'k-', label='Ideal total noise', alpha=0.2)
plt.legend(fontsize=10, ncol=2, loc='upper center')
plt.xlabel(r'Measured mean signal ($e^-/s$)', fontsize=16)
plt.ylabel('Noise-to-signal ratio', fontsize=16)
plt.yscale('log')
plt.xscale('log')
plt.xlim(3E2, 3E5)
plt.ylim(0.003, 2)
plt.show()

In [ ]:
# Get index of star with flux closest to 27109.6
flux_target = 13581.15
# flux_target = 27109.6
idx = (np.abs(light_curve_means - flux_target)).argmin()
print("Closest flux index: ", idx)
print(light_curve_errs[idx])
print(positions[idx])

In [ ]:
print(defect_data_2[6830:6840,3380:3390])
print(defect_data_2[3380:3390,6830:6840])

In [ ]:
# Just keep stuff with light_curve_mean above 1000
light_curve_errs = light_curve_errs[light_curve_means > 1000]
light_curves_2 = light_curves_2[light_curve_means > 1000]
light_curve_means = light_curve_means[light_curve_means > 1000]
max_index = np.argsort(light_curve_errs)[-6]
print(light_curve_errs[max_index], light_curve_means[max_index])
plt.plot(light_curves_2[max_index], 'o')
plt.show()

In [ ]:
print(positions)